# Задание 8

последняя задача позапускать какую-нибудь архитектуру детекции или сегментации

### YOLO - You Only Look Once

#### Библиотеки

In [9]:
import cv2
import numpy as np

import time
import sys
import os

#### Конфигурация сети

Основные настройки

In [10]:
CONFIDENCE = 0.5
SCORE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5

# конфигурационный файл сети
config_path = "yolov3.cfg"
# файл весов сети YOLO
weights_path = "yolov3.weights"

# метки классов
labels = open("listofnames.names").read().strip().split("\n")
# список цветов для выделения объектов
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

Загрузка сети YOLO

In [11]:
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

#### Пример работы с изображением

Подготовка изображения

In [12]:
path_name = "images/food.jpg"
image = cv2.imread(path_name)
file_name = os.path.basename(path_name)
filename, ext = file_name.split(".")

Нормализация изображения для подачи нейронной сети

In [13]:
h, w = image.shape[:2]
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

Измененная форма изображения

In [14]:
print("image.shape:", image.shape)
print("blob.shape:", blob.shape)

image.shape: (480, 640, 3)
blob.shape: (1, 3, 416, 416)


#### Прогнозирование

In [15]:
# устанавливает blob как вход сети
net.setInput(blob)
# получаем имена всех слоев
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
    
# прямая связь (вывод) и получение выхода сети
# измерение времени для обработки в секундах
start = time.perf_counter()
layer_outputs = net.forward(ln)
time_took = time.perf_counter() - start
print(f"Потребовалось: {time_took:.2f}s")

Потребовалось: 1.16s


Получением выделения обнаруженных объектов, метку класса и достоверность обнаружения

In [16]:
font_scale = 1
thickness = 1
boxes, confidences, class_ids = [], [], []
# перебираем каждый из выходов слоя
for output in layer_outputs:
    # перебираем каждое обнаружение объекта
    for detection in output:
        # извлекаем метку класса и достоверность объекта
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        # осталяем объекты с нужной достоверностью
        if confidence > CONFIDENCE:
            # YOLO возвращает координаты центра найденного объекта
            # поэтому нужно масштабировать выделение объекта на фото
            box = detection[:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype("int")
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            # обновляем список полученных детекций
            # выделением, меткой, достоверностью
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

Отрисовка полученных результатов на изображении

In [17]:
# используется non maximum suppression
idxs = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_THRESHOLD, IOU_THRESHOLD)

font_scale = 1
thickness = 1

# если хотя бы одно обнаружение есть
if len(idxs) > 0:
    # проходим по обнаружениям
    for i in idxs.flatten():
        # извлекаем координаты выделения
        x, y = boxes[i][0], boxes[i][1]
        w, h = boxes[i][2], boxes[i][3]
        # отрисовываем прямоугольник на изображении
        color = [int(c) for c in colors[class_ids[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
        text = f"{labels[class_ids[i]]}: {confidences[i]:.2f}"
        # рассчитываем координаты для отрисовки метки класса
        (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
        text_offset_x = x
        text_offset_y = y - 5
        box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
        overlay = image.copy()
        cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
        # добавляем непрозрачный фон
        image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
        # помещаем текст метки на созданное поле
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=font_scale, color=(0, 0, 0), thickness=thickness)

Сохраняем полученное изображение

In [18]:
cv2.imwrite("output/" + filename + "_yolo3." + ext, image)

True